In [64]:
#encoding:utf8
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [59]:
df = pd.read_excel("testcov.xlsx",sheet_name="data")
df = df.set_index("Date").sort_values("Date")
df

,300ETF,500ETF
Date,,
2018-11-05,-1.278620,-0.505263
2018-11-06,-0.632530,0.190436
2018-11-07,-0.818430,-0.950370
2018-11-08,-0.091687,-0.660981
2018-11-09,-1.254206,0.042928
2018-11-12,1.115242,2.617464
2018-11-13,1.041667,1.986201
2018-11-14,-1.212856,-0.451005
2018-11-15,1.319828,1.771005


In [60]:
df_cov = df.rolling(window=10).cov()
df_cov

300ETF    500ETF
Date                                 
2018-11-05 300ETF       NaN       NaN
           500ETF       NaN       NaN
2018-11-06 300ETF       NaN       NaN
           500ETF       NaN       NaN
2018-11-07 300ETF       NaN       NaN
           500ETF       NaN       NaN
2018-11-08 300ETF       NaN       NaN
           500ETF       NaN       NaN
2018-11-09 300ETF       NaN       NaN
           500ETF       NaN       NaN
2018-11-12 300ETF       NaN       NaN
           500ETF       NaN       NaN
2018-11-13 300ETF       NaN       NaN
           500ETF       NaN       NaN
2018-11-14 300ETF       NaN       NaN
           500ETF       NaN       NaN
2018-11-15 300ETF       NaN       NaN
           500ETF       NaN       NaN
2018-11-16 300ETF  1.076911  1.118632
           500ETF  1.118632  1.553121
2018-11-19 300ETF  1.050550  0.922734
           500ETF  0.922734  1.465292
2018-11-20 300ETF  1.538241  1.647488
           500ETF  1.647488  2.482116
2018-11-21 300ETF  1.468902  1.536362
           500ETF  1.536362  2.310627
2018-11-22 300ETF  1.476987  1.536303
           500ETF  1.536303  2.205434
2018-11-23 300ETF  2.035289  2.770323
           500ETF  2.770323  4.401961
...                     ...       ...
2018-12-10 300ETF  1.806788  2.097853
           500ETF  2.097853  2.638854
2018-12-11 300ETF  1.814853  2.144528
           500ETF  2.144528  2.733172
2018-12-12 300ETF  1.588546  1.895463
           500ETF  1.895463  2.489374
2018-12-13 300ETF  1.726552  1.885791
           500ETF  1.885791  2.229252
2018-12-14 300ETF  2.018645  2.309073
           500ETF  2.309073  2.779376
2018-12-17 300ETF  1.179475  1.270216
           500ETF  1.270216  1.507441
2018-12-18 300ETF  1.237579  1.249415
           500ETF  1.249415  1.435642
2018-12-19 300ETF  1.275026  1.307747
           500ETF  1.307747  1.525039
2018-12-20 300ETF  0.999783  0.966833
           500ETF  0.966833  1.329116
2018-12-21 300ETF  1.074031  0.976244
           500ETF  0.976244  1.318192
2018-12-24 300ETF  1.087137  0.998288
           500ETF  0.998288  1.355050
2018-12-25 300ETF  1.021304  0.902976
           500ETF  0.902976  1.269414
2018-12-26 300ETF  0.944776  0.886785
           500ETF  0.886785  1.266442
2018-12-27 300ETF  0.356979  0.326462
           500ETF  0.326462  0.858126
2018-12-28 300ETF  0.303241  0.198123
           500ETF  0.198123  0.631482

[80 rows x 2 columns]

In [62]:
df_one_cov = df_cov.loc['2018-11-16',::]
np.matrix(df_one_cov.values)

matrix([[1.076911  , 1.11863163],
        [1.11863163, 1.55312086]])

In [74]:
#计算资产组合的风险值
def calc_risk_portfolio(weight,cov_matrix):
    weight = np.matrix(weight)
    sigma = np.sqrt(weight*cov_matrix*weight.T)
    MRC = cov_matrix*weight.T/sigma
    RC = np.multiply(MRC,weight.T)
    return RC

#定义优化问题的目标函数
def risk_objective(x_weight,args):
    x_weight = np.matrix(x_weight)
    cov_matrix = args[0]
    RC_target_ratio = args[1]
    sigma_portfolio = np.sqrt(x_weight*cov_matrix*x_weight.T)
    
    RC_target = np.asmatrix(np.multiply(sigma_portfolio,RC_target_ratio))
    RC_real   = calc_risk_portfolio(x_weight,cov_matrix)
    sum_squared_diff = sum(np.square(RC_real-RC_target.T))[0][0]
    return sum_squared_diff

#优化问题的第一个约束条件
def cons_1(x_weight):
    return np.sum(x_weight)-1.0

#优化问题的第二个约束条件
def cons_2(x_weight):
    return x_weight

#根据资产预期目标风险贡献度来计算各资产的权重
def calc_weight_portfolio(RC_target_ratio,weight_default,cov_matrix):
    cons = ({'type':'eq','fun':cons_1},{'type':'ineq',"fun":cons_2})
    res = minimize(risk_objective,weight_default,args=[cov_matrix,RC_target_ratio],method='SLSQP',constraints=cons,options={'disp':True})
    weight_final = np.asmatrix(res.x)
    return weight_final


RC_target_ratio = [0.5,0.5]
weight_default = [0.5,0.5]
cov_matrix = np.matrix(df_one_cov.values)
calc_weight_portfolio(RC_target_ratio,weight_default,cov_matrix)



Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1.02609806618e-09
            Iterations: 2
            Function evaluations: 9
            Gradient evaluations: 2


matrix([[0.54566293, 0.45433707]])

## 引入包

In [99]:
#encoding:utf8
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from datetime import datetime
from WindPy import w

w.start()

.ErrorCode=0
.Data=[Already conntected!]

## 读取数据

In [176]:
# start_date = '2005-01-01'
start_date = '20050101'
end_date = '20181228'

#沪深300、中证500、恒生指数、标普500、日经225、ICE布油连续、COMEX黄金连续
# fields = ['000300.SH','000905.SH','HSI.HI','SPX.GI','N225.GI','B00.IPE','GC00.CMX']
fields = ['000300.SH','HSI.HI','SPX.GI','B00.IPE','GC00.CMX']
indict = ','.join(fields)

#获取各个资产的收益率数据
def get_revenue_data(indict,start_date,end_date):
    wsd_data = w.wsd(indict,"pct_chg",start_date,end_date,"PriceAdj=F")
    df = pd.DataFrame(np.array(wsd_data.Data).T,index=wsd_data.Times,columns=wsd_data.Codes)
    df = df.fillna(0)
    df.index = pd.to_datetime(df.index)
    return df.sort_index()

#获取交易日期序列,频率： {周：W；月：M；季：Q；年：Y}
def get_td_series(start_date,end_date,period):
    w_data = w.tdays(start_date,end_date,"Period="+period)    
    return [x.strftime('%Y-%m-%d') for x in w_data.Data[0]]
 

#获取偏移日期
def get_offset_date(n,t_date,period=""):
    w_data = w.tdaysoffset(n, t_date, "Period="+period)
    return w_data.Data[0][0].strftime('%Y-%m-%d')


# df = get_revenue_data(indict,start_date,end_date)
get_td_series(start_date,end_date,"Q")
# get_offset_date(-10,end_date,"Q")
# df.index


['2005-03-31',
 '2005-06-30',
 '2005-09-30',
 '2005-12-30',
 '2006-03-31',
 '2006-06-30',
 '2006-09-29',
 '2006-12-29',
 '2007-03-30',
 '2007-06-29',
 '2007-09-28',
 '2007-12-28',
 '2008-03-31',
 '2008-06-30',
 '2008-09-26',
 '2008-12-31',
 '2009-03-31',
 '2009-06-30',
 '2009-09-30',
 '2009-12-31',
 '2010-03-31',
 '2010-06-30',
 '2010-09-30',
 '2010-12-31',
 '2011-03-31',
 '2011-06-30',
 '2011-09-30',
 '2011-12-30',
 '2012-03-30',
 '2012-06-29',
 '2012-09-28',
 '2012-12-31',
 '2013-03-29',
 '2013-06-28',
 '2013-09-30',
 '2013-12-31',
 '2014-03-31',
 '2014-06-30',
 '2014-09-30',
 '2014-12-31',
 '2015-03-31',
 '2015-06-30',
 '2015-09-30',
 '2015-12-31',
 '2016-03-31',
 '2016-06-30',
 '2016-09-30',
 '2016-12-30',
 '2017-03-31',
 '2017-06-30',
 '2017-09-29',
 '2017-12-29',
 '2018-03-30',
 '2018-06-29',
 '2018-09-28',
 '2018-12-28']

## 计算组合资产权重

In [ ]:
#计算资产组合的风险值
def calc_risk_portfolio(weight,cov_matrix):
    weight = np.matrix(weight)
    sigma = np.sqrt(weight*cov_matrix*weight.T)
    MRC = cov_matrix*weight.T/sigma
    RC = np.multiply(MRC,weight.T)
    return RC

#定义优化问题的目标函数
def risk_objective(x_weight,args):
    x_weight = np.matrix(x_weight)
    cov_matrix = args[0]
    RC_target_ratio = args[1]
    sigma_portfolio = np.sqrt(x_weight*cov_matrix*x_weight.T)
    
    RC_target = np.asmatrix(np.multiply(sigma_portfolio,RC_target_ratio))
    RC_real   = calc_risk_portfolio(x_weight,cov_matrix)
    sum_squared_diff = sum(np.square(RC_real-RC_target.T))[0][0]
    return sum_squared_diff

#优化问题的第一个约束条件
def cons_1(x_weight):
    return np.sum(x_weight)-1.0

#优化问题的第二个约束条件
def cons_2(x_weight):
    return x_weight

#根据资产预期目标风险贡献度来计算各资产的权重
def calc_weight_portfolio(RC_target_ratio,weight_default,cov_matrix):
    cons = ({'type':'eq','fun':cons_1},{'type':'ineq',"fun":cons_2})
    res = minimize(risk_objective,weight_default,args=[cov_matrix,RC_target_ratio],method='SLSQP',constraints=cons,options={'disp':True})
    weight_final = np.asmatrix(res.x)
    return weight_final



In [187]:
start_date2 = '2005-12-30'
series_q = get_td_series(start_date2,end_date,"Q")
series_d = get_td_series(start_date2,end_date,"D")

d1 = '2016-03-31'
d1_pre = get_offset_date(-1,d1,period="Y")
df.index = pd.to_datetime(df.index)

# df = df[(df.index>datetime.strptime(d1_pre,"%Y-%m-%d")) & (df.index<=datetime.strptime(d1,"%Y-%m-%d"))]
weight = []
for td in series_d:
    if td in series_q:
        

2005-12-30
2006-03-31
2006-06-30
2006-09-29
2006-12-29
2007-03-30
2007-06-29
2007-09-28
2007-12-28
2008-03-31
2008-06-30
2008-09-26
2008-12-31
2009-03-31
2009-06-30
2009-09-30
2009-12-31
2010-03-31
2010-06-30
2010-09-30
2010-12-31
2011-03-31
2011-06-30
2011-09-30
2011-12-30
2012-03-30
2012-06-29
2012-09-28
2012-12-31
2013-03-29
2013-06-28
2013-09-30
2013-12-31
2014-03-31
2014-06-30
2014-09-30
2014-12-31
2015-03-31
2015-06-30
2015-09-30
2015-12-31
2016-03-31
2016-06-30
2016-09-30
2016-12-30
2017-03-31
2017-06-30
2017-09-29
2017-12-29
2018-03-30
2018-06-29
2018-09-28
2018-12-28


In [201]:
import pandas as pd
import json

item = """
{"data":[{"date":"20180928","standard":"0.00","syl":"0.00"},{"date":"20181008","standard":"-4.30","syl":"-0.12"},{"date":"20181009","standard":"-4.37","syl":"-0.12"},{"date":"20181010","standard":"-4.57","syl":"-0.01"},{"date":"20181011","standard":"-9.15","syl":"-0.74"},{"date":"20181012","standard":"-7.80","syl":"-0.39"},{"date":"20181015","standard":"-9.08","syl":"-0.85"},{"date":"20181016","standard":"-9.83","syl":"-1.35"},{"date":"20181017","standard":"-9.32","syl":"-1.06"},{"date":"20181018","standard":"-11.47","syl":"-1.52"},{"date":"20181019","standard":"-8.84","syl":"-0.52"},{"date":"20181022","standard":"-4.90","syl":"1.00"},{"date":"20181023","standard":"-7.43","syl":"0.19"},{"date":"20181024","standard":"-7.29","syl":"0.19"},{"date":"20181025","standard":"-7.11","syl":"0.50"},{"date":"20181026","standard":"-7.71","syl":"-0.11"},{"date":"20181029","standard":"-10.53","syl":"-2.21"},{"date":"20181030","standard":"-9.56","syl":"-2.25"},{"date":"20181031","standard":"-8.29","syl":"-1.38"},{"date":"20181101","standard":"-7.61","syl":"-1.21"},{"date":"20181102","standard":"-4.32","syl":"1.64"},{"date":"20181105","standard":"-5.12","syl":"1.37"},{"date":"20181106","standard":"-5.69","syl":"0.92"},{"date":"20181107","standard":"-6.31","syl":"0.41"},{"date":"20181108","standard":"-6.57","syl":"-0.08"},{"date":"20181109","standard":"-7.89","syl":"-0.79"},{"date":"20181112","standard":"-6.80","syl":"1.05"},{"date":"20181113","standard":"-5.86","syl":"1.68"},{"date":"20181114","standard":"-6.80","syl":"0.79"},{"date":"20181115","standard":"-5.71","syl":"2.89"},{"date":"20181116","standard":"-5.27","syl":"2.90"},{"date":"20181119","standard":"-4.20","syl":"3.58"},{"date":"20181120","standard":"-6.41","syl":"1.06"},{"date":"20181121","standard":"-6.18","syl":"0.45"},{"date":"20181122","standard":"-6.53","syl":"0.39"},{"date":"20181123","standard":"-8.59","syl":"-1.30"},{"date":"20181126","standard":"-8.65","syl":"-1.25"},{"date":"20181127","standard":"-8.77","syl":"-1.13"},{"date":"20181128","standard":"-7.56","syl":"-0.53"},{"date":"20181129","standard":"-8.76","syl":"-1.58"},{"date":"20181130","standard":"-7.74","syl":"-1.27"},{"date":"20181203","standard":"-5.17","syl":"0.56"},{"date":"20181204","standard":"-4.98","syl":"0.70"},{"date":"20181205","standard":"-5.43","syl":"0.59"},{"date":"20181206","standard":"-7.48","syl":"-0.98"},{"date":"20181207","standard":"-7.48","syl":"-1.10"},{"date":"20181210","standard":"-8.55","syl":"-1.58"},{"date":"20181211","standard":"-8.11","syl":"-1.38"},{"date":"20181212","standard":"-7.80","syl":"-1.24"},{"date":"20181213","standard":"-6.37","syl":"-0.59"},{"date":"20181214","standard":"-7.94","syl":"-1.31"},{"date":"20181217","standard":"-8.07","syl":"-1.36"},{"date":"20181218","standard":"-9.03","syl":"-1.93"},{"date":"20181219","standard":"-10.11","syl":"-2.54"},{"date":"20181220","standard":"-10.80","syl":"-2.48"},{"date":"20181221","standard":"-11.91","syl":"-3.05"},{"date":"20181224","standard":"-11.65","syl":"-2.88"},{"date":"20181225","standard":"-12.26","syl":"-3.12"},{"date":"20181226","standard":"-12.70","syl":"-3.27"},{"date":"20181227","standard":"-13.04","syl":"-3.58"},{"date":"20181228","standard":"-12.45","syl":"-3.51"},{"date":"20190102","standard":"-13.65","syl":"-3.90"},{"date":"20190103","standard":"-13.78","syl":"-4.12"},{"date":"20190104","standard":"-11.72","syl":"-3.19"}],"isList":"1","listSize":"64","message":"ok","result":"0"}
"""
df = pd.DataFrame(json.loads(item.strip())['data'])
df.to_excel("portfolio.xlsx",sheet_name="data")